# Color Coding

**Inhalt:** Numerische Werte farblich darstellen

**Nötige Skills:** keine

**Lernziele:**
- Generelle Kenntnisse über Farben im Web
- Colormaps benutzen und selbst kreieren

## Das Beispiel

Eine Liste von Ländern mit ihrer Grösse und diversen Eigenschaften.

Quelle: Weltbank (https://data.worldbank.org/indicator)

## Vorbereitung

In [ ]:
import pandas as pd

In [ ]:
import numpy as np

In [ ]:
from IPython.display import Image

In [ ]:
pd.set_option("display.max_colwidth", 150)

## Daten laden

In [ ]:
path = "dataprojects/Worldbank/worldbank_countries.xlsx"

In [ ]:
df = pd.read_excel(path)

In [ ]:
df.head(3)

### Variablenbeschrieb

In [ ]:
df_vars = pd.read_excel(path, sheet_name='VARIABLES')

In [ ]:
df_vars

Doch bevor wir beginnen... etwas Theorie über Farben im Web und in Python.

## Farben im Web


Farben sind... auch nichts anderes als Zahlen. Es gibt verschiedene Schemas, um Farben zu codieren. Das gängigste im Internet ist **RGB**.
- rot-Intensität
- grün-Intensität
- blau-Intensität

Jede Farbe setzt sich aus drei Komponenten zusammen, rot, grün, blau. Manchmal kommt auch noch eine vierte Komponente dazu, A. Das steht für Alpha, den Transparenzwert.

Man kann RGB-Farben in verschiedenen Zahlensystemen angeben. Zwei oft verwendete und äquivalente Systeme sind:
- In Dezimalzahlen zwischen 0 und 255. Zum Beispiel so: `rgb(102, 153, 0)`
- In Hexadezimalzahlen zwischen 0 und 255: Zum Beispiel so: `#669900`

Um Farben und die dazu passenden Codes zu explorieren, eignet sich der [Color Picker](https://www.w3schools.com/colors/colors_picker.asp) von W3C

In [ ]:
Image("dataprojects/Worldbank/Colorpicker.png")

## Farben in Python und Pandas

Damit wir mit Farben arbeiten können, müssen wir typischerweise Bibliotheken aus `matplotlib` importieren:

In [ ]:
import matplotlib.colors as mcolors

In [ ]:
import matplotlib.pyplot as plt

### Farben spezifizieren

Eine Farbe kann dabei auf verschiedene Arten erstellt werden.

Zum Beispiel dieses schöne **<span style="background-color: #228b22; color: white; padding=1">&nbsp; Grün &nbsp;</span>** hier:

- als Hexadezimalzahl

In [ ]:
c = "#669900"

- als Tupel von Dezimalzahlen (jeweils zwischen 0 und 1, nicht zwischen o und 255)

In [ ]:
c = (0.133, 0.545, 0.133)

- als Dezimalzahlen-Tupel, inkl. Alphawert

In [ ]:
c = (0.133, 0.545, 0.133, 1)

- als eine von den vordefinierten Farben: https://matplotlib.org/stable/gallery/color/named_colors.html

In [ ]:
c = "forestgreen"

In [ ]:
Image("dataprojects/Worldbank/css-colors.png")

Erkennt matplotlib die Farbe anhand einer der obigen Spezifikationsarten, stehen diverse Funktionen zur Verfügung.

### Farben konvertieren

Die naheliegendste Funktion: Den Farbcode für eine bestimmte Farbe anzeigen:

In [ ]:
# Als Hex-Code
mcolors.to_hex(c)

In [ ]:
# Als Dezimal-RGB
mcolors.to_rgb(c)

In [ ]:
# Als Dezimal-RGB, inkl. Alpha-Wert
mcolors.to_rgba(c)

### Farben mischen

Wenn man versteht, dass eine Farbe auch nur eine Kombination von drei Zahlen ist, dann versteht man auch, wie man zwei Farben mischen kann.

Man bildet einfach für jede der drei Komponenten den Mittelwert. Bzw man bewegt sich um einen gewissen Betrag vorwärts auf einem dreidimensionalen Vektor zwischen den beiden Zahlen: man interpoliert.

**Konzeptionell funktioniert das so:** Mischen von rot und blau

- Als Farbe 1 definieren wir ein **<span style="background-color: #0066ff; color: white; padding=1">&nbsp; Blau &nbsp;</span>**

In [ ]:
c1 = (0.0, 0.4, 1.0)

- Als Farbe 2 definieren wir ein **<span style="background-color: #cc0000; color: white; padding=1">&nbsp; Rot &nbsp;</span>**

In [ ]:
c2 = (0.8, 0.0, 0.0)

- Für die Mischung berechnen wir den Mittelwert jedes Elements im Tupel:

In [ ]:
d = 0.5

In [ ]:
c3 = tuple(k1 + (k2 - k1) * d for k1, k2 in zip(c1, c2))
c3

Das Ergebnis: Es resultiert ein **<span style="background-color: #663380; color: white; padding=1">&nbsp; Violett &nbsp;</span>**.

### Interpolieren

Wenn wir nun verschieden lange «Strecken» zwischen **<span style="background-color: #0066ff; color: white; padding=1">&nbsp; Blau &nbsp;</span>** und **<span style="background-color: #cc0000; color: white; padding=1">&nbsp; Rot &nbsp;</span>** zurücklegen wollen, können wir verschieden lange Schritte interpolieren:

In [ ]:
d_list = [0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1]

In [ ]:
d_codes = [tuple(k1 + (k2 - k1) * d for k1, k2 in zip(c1, c2)) for d in d_list]

In [ ]:
df_misch = pd.DataFrame({
    'Distanz': d_list,
    'Farbcode': d_codes
})
df_misch

In [ ]:
df_misch['y'] = 1
df_misch.plot(
    kind='bar',
    x='Distanz',
    y='y',
    color=df_misch['Farbcode'],
    figsize=(8, 1),
    legend=False
)

In der Praxis ist uns das manuelle Mischen von Farben aber zu umständlich. Besser, wir benutzen die Funktionen, die uns Pandas / matplotlib dafür zur Verfügung stellt! Introducing: Colormaps!

## Colormaps

Colormaps sind sehr praktisch: Sie nehmen die Interpolation automatisch für uns vor.

Oder, allgemeiner gesagt: Sie leisten eine **Zuordnung von numerischen Werten zu Farben**.

### Das Konzept

Um diese Zuordnung zu machen, brauchen wir jeweils zwei Angaben:

1. die **Norm**: Der Range von Zahlen, die wir zuordnen möchten
1. den **Mapper**: Ein Farbschema, in das diese Zahlen übersetzt werden sollen.

Konkret könnte das zB so aussehen:

1. Wir wollen Zahlen zwischen 20 und 30 einer Farbe zuordnen

In [ ]:
norm = mcolors.Normalize(vmin=20, vmax=30)

2. Wir wollen, dass 20 rot ist und 30 blau. Dafür gibt es die vordefinierte Colormap `RdBu'

In [ ]:
mapper = plt.cm.ScalarMappable(norm=norm, cmap='RdBu')

In diesen Mapper können wir nun eine beliebige Zahl zwischen 20 und 30 eingeben:

In [ ]:
mapper.to_rgba(24)

Analog zu vorher können wir das mit allen Zahlen zwischen 20 und 30 machen:

In [ ]:
c_list = [20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30]
c_codes = [mapper.to_rgba(c) for c in c_list]

In [ ]:
df_misch = pd.DataFrame({
    'Zahl': c_list,
    'Farbcode': c_codes
})
df_misch

In [ ]:
df_misch['y'] = 1
df_misch.plot(
    kind='bar',
    x='Zahl',
    y='y',
    color=df_misch['Farbcode'],
    figsize=(8, 1),
    legend=False
)

### Vordefinierte Colormaps

In matplotlib gibt es eine Reihe von vordefinierten Colormaps: https://matplotlib.org/stable/gallery/color/colormap_reference.html

Wenn wir in einem Chart einen Wert farblich codieren wollen, können wir diese Colormaps verwenden.

Hier eine Auswahl davon:

In [ ]:
Image("dataprojects/Worldbank/colormaps.png")

Kehren wir nun zürich zu unseren Beispieldaten:

In [ ]:
df.head(2)

### Anwendungsbeispiel: Scatterplot

Wir basteln uns einen Scatterplot nach dem folgenden Prinzip:
- x-Achse: GDP pro Kopf
- y-Achse: Lebenserwartung
- Punktgrösse: Bevölkerung
- Farbe: CO2-Emissionen pro Kopf

Dank der Option `colormap=` geht das in der Plot-Funktion für Scatterplots ganz einfach.

In [ ]:
df.plot(
    kind='scatter',
    x='GDP per Capita',
    y='Life Expectancy',
    s=(df['Population'] / 1000) ** 0.5, # wir müssen die Grössen umrechnen, so dass sie Platz haben
    c='CO2 Emissions per Capita', # c stands for color
    edgecolors='grey',
    colormap='YlOrRd',
    alpha=0.8,
    figsize=(12,8),
    title="CO2-Emissionen pro Kopf für verschiedene Länder"
)

### Anwendungsbeispiel: Barchart

Wir können auch in einem Barchart farbcodierte Informationen einfliessen lassen, wenn wir das sinnvoll finden.

- x-Achse: Namen der zwanzig reichsten Länder
- y-Achse: Bevölkerungsgrösse
- Farbgebung: CO2-Emissonen pro Kopf

Leider funktioniert die Farbgebung hier nicht automatisch, wir müssen die Farbcodes selbst generieren.

In [ ]:
# Wir wählen unsere 10 Länder aus
df_temp = df.sort_values('GDP per Capita', ascending=False).head(20).sort_values('GDP per Capita')

# Liste der Farben: Wo liegen der Minimal- und Maximalwert bei den Emissionen?
min_emissions = df_temp['CO2 Emissions per Capita'].min()
max_emissions = df_temp['CO2 Emissions per Capita'].max()

# Anhand des Min und Max: Norm definieren
norm = mcolors.Normalize(vmin=min_emissions, vmax=max_emissions)

# Colormap auswählen
mapper = plt.cm.ScalarMappable(norm=norm, cmap='YlOrRd')

# Die CO2-Werte der 10 Länder durch den Mapper durchlaufen lassen
colors = [mapper.to_rgba(rate) for rate in df_temp['CO2 Emissions per Capita']]

Nun haben wir zehn Farbcodes für zehn Länder:

In [ ]:
colors

Das Plotten selbst ist dann relativ einfach:

In [ ]:
df_temp.plot(
    kind='barh',
    x='Country Name',
    y='GDP per Capita',
    color=colors,
    figsize=(10,8),
    title="CO2-Emissonen der reichsten zwanzig Länder (nach BIP pro Kopf)",
    legend=False
)

## Colormaps selbst definieren

Hier beginnt die Sache Spass zu machen. Denn erstens sind die vordefinierten Colormaps nicht besonders schön, und zweitens wollen wir manchmal selbst definieren können, wie eine Farbzuordnung genau aussehen soll.

Zum Beispiel für einen Text wie diesen hier: (Link zum Warming Potential)

Dazu brauchen wir zwei Dinge:
1. Eine Reihe von Zahlen, welche die «Stopps» entlang der Zahlenrange bilden
1. Eine Reihe von Farben, die an diesen «Stopps» verwendet werden sollen

Wir können zum Beispiel sagen:
- Emissionswerte unter 10 Tonnen pro Kopf sind gut => grün
- Bei 15 Tonnen pro Kopf kommen wir in den gelben Bereich
- Alles über 20 Tonnen ist schlecht => rot

In [ ]:
cvals  = [0, 10, 15, 20, 25]

In [ ]:
colors = ["#206020", "#339933", "#ff9900", "#ff3300", "#990000"]

Diese Zuordnung würde dann so aussehen:

In [ ]:
df_colors = pd.DataFrame({"values": cvals, "colors": colors, "labels": cvals})
df_colors['labels'] = df_colors['labels'].astype(str)
df_colors['values'] = 1
df_colors.plot(kind='bar', x='labels', y='values', color=df_colors['colors'], legend=False, figsize=(6,1))

Nun wollen wir daraus eine kontinuierliche Colormap basteln.

Der Code dafür ist etwas komplizierter als oben. Am besten einfach den ganzen Code copy-pasten, um ihn zu verwenden.

In [ ]:
# Norm erstellen
norm = plt.Normalize(min(cvals),max(cvals))

# Colormap erstellen
tuples = list(zip(map(norm,cvals), colors))
cmap = mcolors.LinearSegmentedColormap.from_list("", tuples)

# Die Norm und die Colormap ergeben den Mapper
mapper = plt.cm.ScalarMappable(norm=norm, cmap=cmap)

Vorschau auf das Ergebnis:

In [ ]:
n = 512

gradient = np.linspace(min(cvals), max(cvals), n)
gradient = np.vstack((gradient, gradient))

ticks = [(val - min(cvals)) / (max(cvals) - min(cvals)) * n for val in cvals]

fig, ax = plt.subplots()
fig.set_size_inches(15, 2) 
ax.imshow(gradient, aspect=15, cmap=plt.get_cmap(cmap))

ax.xaxis.set_ticks(ticks)
ax.axes.set_xticklabels(cvals)

plt.show()

Unsere selbst definierte Colormap (`cmap`) bzw. den Mapper (`mapper`) können wir nun genau so anwenden wie jede der vordefinierten Colormaps.

In [ ]:
# Die CO2-Werte der 10 Länder durch den neuen Mapper durchlaufen lassen
colors = [mapper.to_rgba(rate) for rate in df_temp['CO2 Emissions per Capita']]

In [ ]:
df_temp.plot(
    kind='barh',
    x='Country Name',
    y='GDP per Capita',
    color=colors,
    figsize=(10,8),
    title="CO2-Emissonen der reichsten zwanzig Länder (nach BIP pro Kopf)",
    legend=False
)

**Key takeaways:**
- Pandas / Matplotlib Plots können von selbst einige Farbcodierungen anwenden
- Es ist mit ein paar wenigen Codezeilen möglich, eigene Farbschemen zu kreieren
- Dabei kann man auch ziemlich kreativ werden und/oder eigene, schönere Schemen erstellen.
- Mit Farben sind immer auch Wertungen verbunden. Genau das kann je nach dem gewünscht sein.

## Übung

### 1. Farbschema anwenden

Aus unserer Datensammlung, wählen Sie eine Land-Eigenschaft aus, die Sie farbcodieren möchten.

In [ ]:
df.head(2)

Überlegen Sie sich: Was könnte ein guter Plot sein, um diesen Wert darzustellen?

Mit welchen anderen Eigenschaften lässt sich dieser Wert in Verbindung bringen?

- Charttyp: ...
- x-Achse: ...
- y-Achse: ...
- ggf Grösse: ...
- Farbe: ...

Wählen Sie eine vorgegebene Colormap aus und wenden Sie diese an einem Plot an.

### 2. Farbschema definieren

Überlegen Sie sich nun noch präziser: Welche Aussagen möchten Sie mit der Farbgebung treffen?

Generieren Sie eine eigene Colormap und wenden Sie diese auf Ihren Chart an.